In [ ]:

import os
import re
##########   Refactoring     ###############
with open('./1.txt', 'r', encoding='UTF-8') as file:
    lines = file.readlines()
seen_filenames = set()

for line in lines:

    match = re.match(r'(.+\\35\\)(.+\.py):\d+-\d+ (.+)', line) or re.match(r'(.+\\35\\)(.+\.py):\d+ (.+)', line)
    if match:
        directory, file_name, modification = match.groups()

        if file_name in seen_filenames:
            continue
        else:
            seen_filenames.add(file_name)

            filename, file_extension = os.path.splitext(file_name)

            parts = filename.split("\\")
            front_address = parts[0]
            new_directory = directory.replace('\\1\\', '\\')

            d = new_directory + front_address
            all_filenames = os.listdir(d)
            filename0 = parts[1]
            new_filename = [filename for filename in all_filenames if filename.startswith(filename0)]
            new_filename = new_filename[0].strip("'")
            new_filename = front_address + "/" + new_filename
            new_file_path = os.path.join(new_directory, new_filename[:-3] + '-1.py')
            try:
                os.rename(os.path.join(new_directory, new_filename), new_file_path)
                print(f"File {os.path.join(new_directory, new_filename)} changed to {new_file_path}")
            except FileNotFoundError:
                print(f"File {os.path.join(new_directory, new_filename)} not found，skip this file")
                continue

print("Finished！")



In [ ]:
##########   Reordering and Formatting     ###############

import ast
import hashlib
import os

import astor
import autopep8


def get_sorted_hash(value):
    sorted_value = ''.join(sorted(value))
    return hashlib.sha256(sorted_value.encode()).hexdigest()


def transform_operations_add(node, lines, file_path):
    if isinstance(node, ast.BinOp) and isinstance(node.op, ast.Add):
        left_var = node.left.id if isinstance(node.left, ast.Name) else None
        right_var = node.right.id if isinstance(node.right, ast.Name) else None

        if left_var and right_var:
            hash_left = get_sorted_hash(left_var)
            hash_right = get_sorted_hash(right_var)

            original_code = astor.to_source(node)

            if hash_left < hash_right:
                node.left, node.right = node.right, node.left

                transformed_code = astor.to_source(node).strip()

                if original_code.strip() != transformed_code:
                    new_file_path = file_path[:-3] + '-36.py'
                    # print(new_file_path)
                    os.rename(file_path,new_file_path)
                    return 1


def transform_operations_mult(node, lines, file_path):
    if isinstance(node, ast.BinOp) and isinstance(node.op, ast.Mult):
        left_var = node.left.id if isinstance(node.left, ast.Name) else None
        right_var = node.right.id if isinstance(node.right, ast.Name) else None

        if left_var and right_var:
            hash_left = get_sorted_hash(left_var)
            hash_right = get_sorted_hash(right_var)

            original_code = astor.to_source(node)

            if hash_left < hash_right:
                node.left, node.right = node.right, node.left

                transformed_code = astor.to_source(node).strip()

                if original_code.strip() != transformed_code:
                    new_file_path = file_path[:-3] + '-37.py'
                    # print(new_file_path)
                    os.rename(file_path,new_file_path)
                    return 1

def transform_comparison_LG(node, lines, file_path):
    if isinstance(node, ast.Compare):
        for op in node.ops:
            if isinstance(op, (ast.Gt, ast.Lt)):
                left_var = node.left.id if isinstance(node.left, ast.Name) else None
                right_var = node.comparators[0].id if isinstance(node.comparators[0], ast.Name) else None

                if left_var and right_var:
                    hash_left = get_sorted_hash(left_var)
                    hash_right = get_sorted_hash(right_var)

                    original_code = astor.to_source(node)

                    if hash_left < hash_right:
                        if isinstance(op, ast.Gt):
                            node.left, node.comparators[0] = node.comparators[0], node.left
                            node.ops[0] = ast.Lt()
                        elif isinstance(op, ast.Lt):
                            node.left, node.comparators[0] = node.comparators[0], node.left
                            node.ops[0] = ast.Gt()


                        transformed_code = astor.to_source(node).strip()

                        if original_code.strip() != transformed_code:
                            new_file_path = file_path[:-3] + '-38.py'
                            # print(new_file_path)
                            os.rename(file_path,new_file_path)
                            return 1
def transform_comparison_LGE(node, lines, file_path):
    if isinstance(node, ast.Compare):
        for op in node.ops:
            if isinstance(op, (ast.GtE, ast.LtE)):
                left_var = node.left.id if isinstance(node.left, ast.Name) else None
                right_var = node.comparators[0].id if isinstance(node.comparators[0], ast.Name) else None

                if left_var and right_var:
                    hash_left = get_sorted_hash(left_var)
                    hash_right = get_sorted_hash(right_var)

                    original_code = astor.to_source(node)

                    if hash_left < hash_right:
                        if isinstance(op, ast.GtE):
                            node.left, node.comparators[0] = node.comparators[0], node.left
                            node.ops[0] = ast.LtE()
                        elif isinstance(op, ast.LtE):
                            node.left, node.comparators[0] = node.comparators[0], node.left
                            node.ops[0] = ast.GtE()

                        transformed_code = astor.to_source(node).strip()

                        if original_code.strip() != transformed_code:
                            new_file_path = file_path[:-3] + '-39.py'
                            # print(new_file_path)
                            os.rename(file_path,new_file_path)
                            return 1

if __name__ == "__main__":
   
    folder_paths = [
        "G\APPS_3.5_G",
        "G\APPS_4_G",
        "G\MBPP_3.5_G",
        "G\MBPP_4_G",
        "G\MBPP_Starcoder_G",
        "H\APPS_3.5_H",
        "H\APPS_4_H",
        "H\MBPP_3.5_H",
        "H\MBPP_4_H",
        "H\MBPP_Starcoder_H"

    ]
    for folder_path in folder_paths:
     
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]
        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)
                #rule 37
                for node in ast.walk(tree):
                    flag = transform_operations_add(node, lines, file_path)
                    if flag == 1:
                        break
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    for folder_path in folder_paths:
    
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]

        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)
                #rule 38
                for node in ast.walk(tree):
                    flag = transform_operations_mult(node, lines, file_path)
                    if flag == 1:
                        break
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    for folder_path in folder_paths:
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]

        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)
                # rule 39
                for node in ast.walk(tree):
                    flag = transform_comparison_LG(node, lines, file_path)
                    if flag == 1:
                        break
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

    for folder_path in folder_paths:
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]
        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)
                # rule 40
                for node in ast.walk(tree):
                    flag = transform_comparison_LGE(node, lines, file_path)
                    if flag == 1:
                        break
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    for folder_path in folder_paths:
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]
        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)
                # 41
                select_rules = ['E101', 'E11', 'E121', 'E122', 'E125']
                formatted_code = autopep8.fix_code(original_code, options={'select': select_rules})
                if formatted_code != original_code:
                    new_file_path = file_path[:-3] + '-40.py'
                    # print(new_file_path)
                    os.rename(file_path, new_file_path)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    for folder_path in folder_paths:
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]
        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)

                # 42
                select_rules = ['E126']
                formatted_code = autopep8.fix_code(original_code, options={'select': select_rules})
                if formatted_code != original_code:
                    new_file_path = file_path[:-3] + '-41.py'
                    # print(new_file_path)
                    os.rename(file_path, new_file_path)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    for folder_path in folder_paths:
   
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]
        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)
                # 43
                select_rules = ['E225', 'E226', 'E227', 'E228', 'E241', 'E242', 'E251', 'E252']
                formatted_code = autopep8.fix_code(original_code, options={'select': select_rules})
                if formatted_code != original_code:
                    new_file_path = file_path[:-3] + '-42.py'
                    # print(new_file_path)
                    os.rename(file_path, new_file_path)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    for folder_path in folder_paths:
     
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]
        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)
                # 44
                select_rules = ['E27']
                formatted_code = autopep8.fix_code(original_code, options={'select': select_rules})
                if formatted_code != original_code:
                    new_file_path = file_path[:-3] + '-43.py'
                    # print(new_file_path)
                    os.rename(file_path, new_file_path)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    for folder_path in folder_paths:
  
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]
        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)

                # 45
                select_rules = ['E26', 'E265']
                formatted_code = autopep8.fix_code(original_code, options={'select': select_rules})
                if formatted_code != original_code:
                    new_file_path = file_path[:-3] + '-44.py'
                    # print(new_file_path)
                    os.rename(file_path, new_file_path)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    for folder_path in folder_paths:
     
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]

        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)

                # 46
                select_rules = ['E266']
                formatted_code = autopep8.fix_code(original_code, options={'select': select_rules})
                if formatted_code != original_code:
                    new_file_path = file_path[:-3] + '-45.py'
                    # print(new_file_path)
                    os.rename(file_path, new_file_path)

            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    for folder_path in folder_paths:

        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]
        for file in python_files:
            file_path = os.path.join(folder_path, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    original_code = f.read()
                    lines = original_code.split('\n')
                tree = ast.parse(original_code)

                # 47
                select_rules = ['E123']
                formatted_code = autopep8.fix_code(original_code, options={'select': select_rules})
                if formatted_code != original_code:
                    new_file_path = file_path[:-3] + '-46.py'
                    # print(new_file_path)
                    os.rename(file_path, new_file_path)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")



In [ ]:
#######   number of the applicable transformation     ################
import os

if __name__ == "__main__":
    folder_paths_G = [
        "G\APPS_3.5_G",
        "G\APPS_4_G",
        "G\MBPP_3.5_G",
        "G\MBPP_4_G",
        "G\MBPP_Starcoder_G"

    ]
    folder_paths_H = [
        "H\APPS_3.5_H",
        "H\APPS_4_H",
        "H\MBPP_3.5_H",
        "H\MBPP_4_H",
        "H\MBPP_Starcoder_H"

    ]
    num_G = [[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]
    num_H = [[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]

    num_of_set = 0
    for folder_path in folder_paths_G:
     
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]
        for file in python_files:
            file_path = os.path.join(folder_path, file)
            index = file.count('-')
            if 0<index<=6:
                new_name = os.path.join(folder_path,folder_path[2:]+'_'+str(index),file)
                # print(new_name)
                os.rename(file_path,new_name)
                num_G[num_of_set][index-1]+=1
            elif index>6:
                new_name = os.path.join(folder_path,folder_path[2:]+'_over6',file)
                # print(new_name)
                os.rename(file_path,new_name)
                num_G[num_of_set][6]+=1
        num_of_set+=1
    num_of_set = 0
    for folder_path in folder_paths_H:
     
        python_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]
        for file in python_files:
            file_path = os.path.join(folder_path, file)
            index = file.count('-')
            if 0 < index <= 6:
                new_name = os.path.join(folder_path, folder_path[2:] + '_' + str(index), file)
                # print(new_name)
                os.rename(file_path,new_name)
                num_H[num_of_set][index - 1] += 1
            elif index > 6:
                new_name = os.path.join(folder_path, folder_path[2:] + '_over6', file)
                # print(new_name)
                os.rename(file_path,new_name)
                num_H[num_of_set][6] += 1
        num_of_set += 1
    print(num_G)
    print(num_H)
    num_final = [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]
    for x in range(5):
        for y in range(7):
            num_final[x][y] = min(num_G[x][y], num_H[x][y])
    print(num_final)
   
    C = [
        "_1",
        "_2",
        "_3",
        "_4",
        "_5",
        "_6",
        "_over6",

    ]
    num_of_set = 0

    count = 0
    for folder_path in folder_paths_G:
    
        num_of_class = 0
        for c in C:
            count = 0
            new_folder_path = os.path.join(folder_path, folder_path[2:]+c)
            python_files = [f for f in os.listdir(new_folder_path) if f.endswith('.py')]
            for file in python_files:
                count+=1
                if count> num_final[num_of_set][num_of_class]:
                    remove_path = os.path.join(new_folder_path,file)
                    os.remove(remove_path)
                    # a=1
            num_of_class+=1
        num_of_set+=1
    num_of_set = 0

    count = 0
    for folder_path in folder_paths_H:
        num_of_class = 0
      
        for c in C:
            count=0
            new_folder_path = os.path.join(folder_path, folder_path[2:]+c)
            python_files = [f for f in os.listdir(new_folder_path) if f.endswith('.py')]
            for file in python_files:
                count+=1
                if count> num_final[num_of_set][num_of_class]:
                    remove_path = os.path.join(new_folder_path,file)
                    os.remove(remove_path)
                
            num_of_class += 1
        num_of_set += 1

In [ ]:
######  count lines     ##############

import os

import openpyxl

if __name__ == "__main__":
    folder_paths = [
        "G\MBPP_3.5_G",
        "G\MBPP_4_G",
        "G\APPS_3.5_G",
        "G\APPS_4_G",
        "G\MBPP_Starcoder_G"

    ]
    C = [
        "_1",
        "_2",
        "_3",
        "_4",
        "_5",
        "_6",
        "_over6",

    ]
    results = [0 for _ in range(35)]
    count_set = 0
    for folder_path in folder_paths:
        for c in C:
            count_set += 1
            new_folder_path = os.path.join(folder_path, folder_path[2:]+c)
            python_files = [f for f in os.listdir(new_folder_path) if f.endswith('.py')]
            count_line = 0
            count_file = 0
            for file in python_files:
                file_path = os.path.join(new_folder_path, file)
                try:
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                        original_code = f.readlines()
                        count_file += 1
                        lines = len(original_code)
                        count_line+=lines
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")
            if count_file >0:
                results[count_set-1] = count_line/count_file
            else:
                results[count_set-1] = 0
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = 'Count'
   

    for row_num, result in enumerate(results):
        sheet.cell(row=row_num+1, column=1, value=result)

    workbook.save('count.xlsx')
